# Deel 1 : Productieproces
## Imports, Directories

In [ ]:
# Importeren van module Tools waarin constanten en functies ondergebracht zijn
import sys
import os
dirscripts  = os.path.join(os.path.dirname(os.getcwd()), "Scripts")
sys.path.insert(0, dirscripts)
import Tools
from Tools import np, pd, json, plt, stats, norm, ECDF, FuncFormatter #Ook alle libraries zijn in Tools ondergebracht!

## Read Production Data BRU & STO

In [ ]:
# Create dictionary from input file "master_data.json"
# MSR = Maximum Sustainable Rate
with open(os.path.join(Tools.dirdataprod, 'master_data.json')) as file:
    dicmsr = json.loads(file.read())    

In [ ]:
# Create dataframe "dfbru" dat de productiegegevens van BRU(ssel) bevat
dfbru = Tools.df_from_json_files(Tools.dirbru)
dfbru["City"] = "BRU"
dfbru.head()

In [ ]:
# Create dataframe "dfsto" dat de productiegegevens van STO(ckholm) bevat
dfsto = Tools.df_from_json_files (Tools.dirsto)
dfsto["City"] = "STO"
dfsto.head()

## Clean Production Data BRU & STO

In [ ]:
dfbru = Tools.df_clean(dfbru)
dfsto = Tools.df_clean(dfsto)

## Analyse Production Data BRU & STO

In [ ]:
# Visualize production over past years (for information)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))  # 1 row, 2 columns

#Function for subplots
Tools.plot_production(dfbru, ax1, 'BRU', dicmsr['BRU msr'])
Tools.plot_production(dfsto, ax2, 'STO', dicmsr['STO msr'])

plt.tight_layout()


In [ ]:
# Maintenance dagen verwijderen
dfpbru = dfbru.loc[dfbru["maintenance"] == "No",:].reset_index(drop=True).copy()
dfpsto = dfsto.loc[dfsto["maintenance"] == "No",:].reset_index(drop=True).copy()

In [ ]:
# Informatief histogram na verwijderen maintenance dagen
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
Tools.plot_histogram(dfpbru, "production", axs[0], "BRU")
Tools.plot_histogram(dfpsto, "production", axs[1], "STO")
plt.tight_layout()

### Verdeling bepalen voor productie van 1 dag

In [ ]:
# Parameters (mean & std) bepalen van de Normale verdeling die de "niet-nul" productiedagen fit
# de "niet-nul" productiedagen : dfp.loc[dfp["production"] != 0,:]
dfpbru_not_0_model = stats.norm.fit(dfpbru.loc[dfpbru["production"] != 0,:]["production"])
dfpsto_not_0_model = stats.norm.fit(dfpsto.loc[dfpsto["production"] != 0,:]["production"])


In [ ]:
# Percentage van "nul" productiedagen bepalen
bru_perc_0 = np.mean(dfpbru["production"] == 0)   # dfp["production"] == 0 is een Boolean Series
sto_perc_0 = np.mean(dfpsto["production"] == 0)   # dfp["production"] == 0 is een Boolean Series
print(f"Percentage of zero production BRU: {bru_perc_0 * 100:.2f}%")
print(f"Percentage of zero production STO: {sto_perc_0 * 100:.2f}%")


## Simulatieprogramma

In [ ]:
# Maak een simulatie van het Model voor de productie van 1 dag
SS = 2000
bru_simulation_prod = [Tools.model_prod(bru_perc_0, dfpbru_not_0_model[0], dfpbru_not_0_model[1]) for _ in range(SS)]
sto_simulation_prod = [Tools.model_prod(sto_perc_0, dfpsto_not_0_model[0], dfpsto_not_0_model[1]) for _ in range(SS)]
bru_simulation_prod_df = pd.DataFrame(bru_simulation_prod, columns=['production'])
sto_simulation_prod_df = pd.DataFrame(sto_simulation_prod, columns=['production'])


In [ ]:
# Beide Density Histogrammen in 1 figuur
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

# Plot histograms for BRU
Tools.plot_histogram(dfpbru, "production", axs[0], colorp='b', alphap=0.2, densityp=True)
Tools.plot_histogram(bru_simulation_prod_df, "production", axs[0],title='BRU actual vs. simulated', colorp='r', alphap=0.2, densityp=True)

# Plot histograms for STO
Tools.plot_histogram(dfpsto, "production", axs[1], colorp='b', alphap=0.2, densityp=True)
Tools.plot_histogram(sto_simulation_prod_df, "production", axs[1], title='STO actual vs. simulated', colorp='r', alphap=0.2, densityp=True)

# Enhance the plots
axs[0].legend(['Actual', 'Simulated'])
axs[1].legend(['Actual', 'Simulated'])
plt.tight_layout()

### Simulatie van totale productie over periode van n dagen

In [ ]:
n = 7
SS = 2000
bru_simulation_prodn = [sum([Tools.model_prod(bru_perc_0, dfpbru_not_0_model[0], dfpbru_not_0_model[1]) for _ in range(n)]) for k in range(SS)]
sto_simulation_prodn = [sum([Tools.model_prod(sto_perc_0, dfpsto_not_0_model[0], dfpsto_not_0_model[1]) for _ in range(n)]) for k in range(SS)]
bru_simulation_prodn_df = pd.DataFrame(bru_simulation_prodn, columns=['production'])
sto_simulation_prodn_df = pd.DataFrame(sto_simulation_prodn, columns=['production'])


In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))
Tools.plot_histogram(bru_simulation_prodn_df, "production", axs[0], title=f'BRU simulation {n} days')
Tools.plot_histogram(sto_simulation_prodn_df, "production", axs[1], title=f'STO simulation {n} days')
plt.tight_layout()


## ECDF

In [ ]:
# Berekenen ECDF
bru_ecdf = ECDF(bru_simulation_prodn)
sto_ecdf = ECDF(sto_simulation_prodn)
brumax = n * dicmsr['BRU msr']
stomax = n * dicmsr['STO msr']

print(f"\033[4mECDF BRU: PRODUCTION FOR {n} DAYS - Theoretical maximum production for {n} days is {brumax}\033[0m")
print(f'P(x<25%): {bru_ecdf(brumax * 0.25):.3f} - Between 0 and {brumax * 0.25} hl')
print(f'P(x<50%): {bru_ecdf(brumax * 0.50):.3f} - Between 0 and {brumax * 0.50} hl')
print(f'P(x<75%): {bru_ecdf(brumax * 0.75):.3f} - Between 0 and {brumax * 0.75} hl')
print()
print(f"\033[4mECDF STO: PRODUCTION FOR {n} DAYS - Theoretical maximum production for {n} days is {stomax}\033[0m")
print(f'P(x<25%): {sto_ecdf(stomax * 0.25):.3f} - Between 0 and {stomax * 0.25} hl')
print(f'P(x<50%): {sto_ecdf(stomax * 0.50):.3f} - Between 0 and {stomax * 0.50} hl')
print(f'P(x<75%): {sto_ecdf(stomax * 0.75):.3f} - Between 0 and {stomax * 0.75} hl')
print()

In [ ]:
# Plot ECDF
fig, axs = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns

# Call the plotting function for each subplot
Tools.plot_ecdf(bru_ecdf, axs[0], brumax, f'ECDF for BRU {n} DAYS')
Tools.plot_ecdf(sto_ecdf, axs[1], stomax, f'ECDF for STO {n} DAYS')
plt.tight_layout()  # Adjust layout


## Centrale limietstelling  


In [ ]:
# We use the ECDF in the previous cells and compare it with a Normal CDF. As the number of days increases, we will calculate the ECDF for the production data of
# our case, and for a normal distribution

# Define sample sizes
samples_sizes = [2, 5, 10, 50]

# Define the datasets and their parameters
datasets = [
    {'name': 'BRU', 'perc': bru_perc_0, 'model_params': dfpbru_not_0_model},
    {'name': 'STO', 'perc': sto_perc_0, 'model_params': dfpsto_not_0_model}
]

# Create a figure with enough subplots for both datasets
fig, axes = plt.subplots(2, 4, figsize=(16, 8))  # 2 rows for 2 datasets, 4 columns for sample sizes

for i, dataset in enumerate(datasets):
    for j, days in enumerate(samples_sizes):
        # Generate summed data
        summed_data = Tools.generate_summed_data(dataset['perc'], dataset['model_params'], days)
        # Select the appropriate subplot
        ax = axes[i, j]
        # Use the function to compare ECDF to Normal CDF and plot
        Tools.compare_ecdf_to_normal(summed_data, ax, days)
        # Set subplot title
        ax.set_title(f"{dataset['name']} with n={days}")
        ax.xaxis.set_major_formatter(FuncFormatter(Tools.thousands_formatter))        
# Add a main title for the entire figure
fig.suptitle('ECDF vs Normal CDF for Both Datasets and Different Sample Sizes', fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the main title
